# Correlation Power Analysis (Brier et al. 2004) - Evolution Plot

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import random

import lascar
import numpy as np
import plotly.graph_objects as pgo
from cwtoolbox import CaptureDevice

In [ ]:
class EvolutionOutputMethod(lascar.ScoreProgressionOutputMethod):
    def _finalize(self):
        scores = np.array(next(iter(self.scores.values()))).T
        fig = pgo.Figure()
        for guess, score in zip(self.engines[0]._guess_range, scores):
            fig.add_trace(pgo.Scatter(x=self.steps, y=score, name=str(guess)))
        fig.show()

In [ ]:
capture_device = CaptureDevice.create("CWLITEXMEGA")
capture_device.compile(file=os.path.abspath("../lecture_3/sbox_lookup.c"))
capture_device.flash()
data = capture_device.capture(
    number_of_traces=1000,
    input=lambda _: [random.randint(0, 255)] + 15 * [0],
)

In [ ]:
def selection_function(value, guess):
    return lascar.hamming(lascar.tools.aes.sbox[value["input"][0] ^ guess])


trace = lascar.TraceBatchContainer(data["trace"], data)
engine = lascar.CpaEngine(
    selection_function=selection_function,
    guess_range=range(256),
)

session = lascar.Session(
    trace,
    engine=engine,
    output_method=EvolutionOutputMethod(engine),
    output_steps=range(0, len(data), 5),
    progressbar=False,
)
session.run(batch_size="auto")